In [ ]:
from torchvision import datasets, transforms
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import torch
from tqdm import tqdm
import time

In [ ]:
def cal_distance(x, y):
    #return torch.sum((x-y)**2)**0.5
    #return torch.sum((x-y)**4)**0.5
    #return torch.sum((x-y)**6)**0.5 #best
    #return 1-torch.cosine_similarity(x,y,dim=0) #95.5 k=11
    # return torch.dist(x, y, 6)#best k=6
    return torch.dist(x, y, 2)
    

In [ ]:

# iteration func
def KNN_by_iter(train_x, train_y, test_x, test_y, k,dimension):
    pca=PCA(dimension)
    pca.fit(train_x)
    train_x_reduct=torch.from_numpy(pca.transform(train_x))
    test_x_reduct=torch.from_numpy(pca.transform(test_x))
    
    since = time.time()

    # cal distance
    res = []
    for x in tqdm(test_x_reduct):
        dists = []
        for y in train_x_reduct:
            dists.append(cal_distance(x,y).view(1))
        
        idxs = torch.cat(dists).argsort()[:k]
        res.append(np.bincount(np.array([train_y[idx] for idx in idxs])).argmax())

    print("acc",accuracy_score(test_y, res))

    time_elapsed = time.time() - since
    print('KNN iter training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
        

In [ ]:
def Run_Knn(train_dataset,test_x,test_y,mask_size):
    train_x = []
    train_y = []
    mask_start_x=10
    mask_start_y=10
    for i in range(len(train_dataset)):
        img, target = train_dataset[i]
        img=img.squeeze()
        img[mask_start_x:mask_start_x+mask_size,mask_start_y:mask_start_y+mask_size]=torch.zeros((mask_size,mask_size))
        train_x.append(img.view(-1))
        train_y.append(target)
        if i > 1000:
            break
        
    KNN_by_iter(torch.stack(train_x), train_y, torch.stack(test_x), test_y, 4,0.9)

In [ ]:
if __name__ == "__main__":

    train_dataset = datasets.MNIST(root="./data", transform=transforms.ToTensor(), train=True,download=True)
    test_dataset = datasets.MNIST(root="./data", transform=transforms.ToTensor(), train=False,download=True)

    test_x = [] 
    test_y = []
    for i in range(len(test_dataset)):
        img, target = test_dataset[i]
        img=img.squeeze()      
        test_x.append(img.view(-1))
        test_y.append(target)


    for mask_size in [1,2,3,4,5,6,7,8]:
        Run_Knn(train_dataset,test_x,test_y,mask_size)

    
    
    
